In [ ]:
import re
from nltk.tokenize import TreebankWordTokenizer, WhitespaceTokenizer
from nltk.corpus import stopwords
import nltk
from nltk.stem import PorterStemmer, WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data = open('/content/drive/My Drive/11-0.txt', 'r', encoding='utf8').read()
data

'\ufeffThe Project Gutenberg eBook of Alice’s Adventures in Wonderland, by Lewis Carroll\n\nThis eBook is for the use of anyone anywhere in the United States and\nmost other parts of the world at no cost and with almost no restrictions\nwhatsoever. You may copy it, give it away or re-use it under the terms\nof the Project Gutenberg License included with this eBook or online at\nwww.gutenberg.org. If you are not located in the United States, you\nwill have to check the laws of the country where you are located before\nusing this eBook.\n\nTitle: Alice’s Adventures in Wonderland\n\nAuthor: Lewis Carroll\n\nRelease Date: January, 1991 [eBook #11]\n[Most recently updated: October 12, 2020]\n\nLanguage: English\n\nCharacter set encoding: UTF-8\n\nProduced by: Arthur DiBianca and David Widger\n\n*** START OF THE PROJECT GUTENBERG EBOOK ALICE’S ADVENTURES IN WONDERLAND ***\n\n[Illustration]\n\n\n\n\nAlice’s Adventures in Wonderland\n\nby Lewis Carroll\n\nTHE MILLENNIUM FULCRUM EDITION 3.0\n\n

In [ ]:
data = re.sub("<.*?>", " ", data)
data = re.sub(" +", " ", data)
data = re.sub("\n", " ", data)
data = data[data.find('CHAPTER I. Down the Rabbit-Hole   Alice'):data.find('*** END OF THE PROJECT GUTENBERG')]
data = re.sub('[^A-Za-z0-9 ]', ' ', data)
data = " ".join(data.split())
data

'CHAPTER I Down the Rabbit Hole Alice was beginning to get very tired of sitting by her sister on the bank and of having nothing to do once or twice she had peeped into the book her sister was reading but it had no pictures or conversations in it and what is the use of a book thought Alice without pictures or conversations So she was considering in her own mind as well as she could for the hot day made her feel very sleepy and stupid whether the pleasure of making a daisy chain would be worth the trouble of getting up and picking the daisies when suddenly a White Rabbit with pink eyes ran close by her There was nothing so very remarkable in that nor did Alice think it so very much out of the way to hear the Rabbit say to itself Oh dear Oh dear I shall be late when she thought it over afterwards it occurred to her that she ought to have wondered at this but at the time it all seemed quite natural but when the Rabbit actually took a watch out of its waistcoat pocket and looked at it and 

In [ ]:
chapters = {'Chapter I':data[data.find('Alice was beginning'):data.find('CHAPTER II')],
 'Chapter II':data[data.find('Curiouser and curiouser'):data.find('CHAPTER III')],
 'Chapter III':data[data.find('They were indeed'):data.find('CHAPTER IV')],
 'Chapter IV':data[data.find('It was the White Rabbit'):data.find('CHAPTER V')],
 'Chapter V':data[data.find('The Caterpillar and Alice looked'):data.find('CHAPTER VI')],
 'Chapter VI':data[data.find('For a minute or two she'):data.find('CHAPTER VII')],
 'Chapter VII':data[data.find('There was a table set out'):data.find('CHAPTER VIII')],
 'Chapter VIII':data[data.find('A large rose tree stood'):data.find('CHAPTER IX')],
 'Chapter IX':data[data.find('You can t think how glad'):data.find('CHAPTER X')],
 'Chapter X':data[data.find('The Mock Turtle sighed deeply'):data.find('CHAPTER XI')],
 'Chapter XI':data[data.find('The King and Queen of Hearts'):data.find('CHAPTER XII')],
 'Chapter XII':data[data.find('Here cried Alice'):data.find('THE END')],
 }

In [ ]:
for key in chapters.keys():
  chapters[key] = chapters[key].lower()
  chapters[key] = re.sub(r"[^\w\s]", "", chapters[key])
  chapters[key] = " ".join(chapters[key].split())

In [ ]:
data = data.lower()

In [ ]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
for key in chapters.keys():
    tokens = TreebankWordTokenizer().tokenize(chapters[key])
    stop_words = stopwords.words("english") 
    tokens = [token for token in tokens if token not in stop_words]
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    text = " ".join(tokens)
    chapters[key] = text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [ ]:
chapters

{'Chapter I': 'alice beginning get tired sitting sister bank nothing twice peeped book sister reading picture conversation use book thought alice without picture conversation considering mind well could hot day made feel sleepy stupid whether pleasure making daisy chain would worth trouble getting picking daisy suddenly white rabbit pink eye ran close nothing remarkable alice think much way hear rabbit say oh dear oh dear shall late thought afterwards occurred ought wondered time seemed quite natural rabbit actually took watch waistcoat pocket looked hurried alice started foot flashed across mind never seen rabbit either waistcoat pocket watch take burning curiosity ran across field fortunately time see pop large rabbit hole hedge another moment went alice never considering world get rabbit hole went straight like tunnel way dipped suddenly suddenly alice moment think stopping found falling deep well either well deep fell slowly plenty time went look wonder going happen next first trie

In [ ]:
chapters_without_alice = chapters.copy()
for key in chapters_without_alice.keys():
  chapters_without_alice[key] = re.sub("alice", "", chapters_without_alice[key])

In [ ]:
vectorizer_tftidf = TfidfVectorizer()
vectorizer_tftidf_fit_transform = vectorizer_tftidf.fit_transform(chapters_without_alice.values())

In [ ]:
names = vectorizer_tftidf.get_feature_names_out()

In [ ]:
for i in range(12):
  first_vector_tfidf_vectorizer = vectorizer_tftidf_fit_transform[i] 
  df = pd.DataFrame(first_vector_tfidf_vectorizer.T.todense(), index=names, columns=["tfidf"]) 
  print(f"\nTop 10 most important words in the Chapter {i+1}:\n")
  print(df.sort_values(by=["tfidf"], ascending=False).head(10))


Top 10 most important words in the Chapter 1:

           tfidf
little  0.173599
bat     0.171262
door    0.154730
way     0.150453
eat     0.143651
rabbit  0.137538
key     0.135790
think   0.127306
like    0.127306
see     0.115733

Top 10 most important words in the Chapter 2:

           tfidf
mouse   0.384872
little  0.192437
pool    0.172634
oh      0.171741
dear    0.157187
cat     0.146618
swam    0.139593
said    0.135838
foot    0.132109
go      0.124518

Top 10 most important words in the Chapter 3:

            tfidf
said     0.391798
mouse    0.391796
dodo     0.301268
prize    0.170525
lory     0.150634
race     0.142104
dry      0.135206
know     0.126758
bird     0.122600
thimble  0.113683

Top 10 most important words in the Chapter 4:

           tfidf
mouse   0.337945
said    0.279891
little  0.199244
rabbit  0.140942
dodo    0.134360
one     0.132830
bill    0.129875
glove   0.128701
oh      0.121801
thing   0.118598

Top 10 most important words in the Chapter 5:

 

So, from the results obtained, we can suggest the following titles for the chapters of "Alice’s Adventures in Wonderland":
- Chapter 1: "Little door";
- Chapter 2: "Little mouse in the pool";
- Chapter 3: "Mouse's story";
- Chapter 4: "Getting to know the rabbit";
- Chapter 5: "Caterpillar, serpent and pigeon";
- Chapter 6: "The house in which the footman, the cat, the baby and the duchess ended up";
- Chapter 7: "Tea party of dormouse, hatter and cat";
- Chapter 8: "The conversation of the queen, the hedgehog and the king with the gardener";
- Chapter 9: "The turtle and the griffin mock the duchess";
- Chapter 10: "The turtle and the griffin mock the lobster";
- Chapter 11: "The king and the hatter at the dormouse court";
- Chapter 12: "The king sits on the jury and talks to the queen's sister about the important rabbit's dreams";

In [ ]:
data_for_verbs = open('/content/drive/My Drive/11-0.txt', 'r').read()
data_for_verbs

'\ufeffThe Project Gutenberg eBook of Alice’s Adventures in Wonderland, by Lewis Carroll\n\nThis eBook is for the use of anyone anywhere in the United States and\nmost other parts of the world at no cost and with almost no restrictions\nwhatsoever. You may copy it, give it away or re-use it under the terms\nof the Project Gutenberg License included with this eBook or online at\nwww.gutenberg.org. If you are not located in the United States, you\nwill have to check the laws of the country where you are located before\nusing this eBook.\n\nTitle: Alice’s Adventures in Wonderland\n\nAuthor: Lewis Carroll\n\nRelease Date: January, 1991 [eBook #11]\n[Most recently updated: October 12, 2020]\n\nLanguage: English\n\nCharacter set encoding: UTF-8\n\nProduced by: Arthur DiBianca and David Widger\n\n*** START OF THE PROJECT GUTENBERG EBOOK ALICE’S ADVENTURES IN WONDERLAND ***\n\n[Illustration]\n\n\n\n\nAlice’s Adventures in Wonderland\n\nby Lewis Carroll\n\nTHE MILLENNIUM FULCRUM EDITION 3.0\n\n

In [ ]:
data_for_verbs = re.sub("<.*?>", " ", data_for_verbs)
data_for_verbs = re.sub(" +", " ", data_for_verbs)
data_for_verbs = re.sub("\n", " ", data_for_verbs)
data_for_verbs = data_for_verbs[data_for_verbs.find('CHAPTER I. Down the Rabbit-Hole   Alice'):data_for_verbs.find('*** END OF THE PROJECT GUTENBERG')]
data_for_verbs = re.sub("[!();?*“‘’”:,—_-]", "", data_for_verbs)
data_for_verbs

'CHAPTER I. Down the RabbitHole   Alice was beginning to get very tired of sitting by her sister on the bank and of having nothing to do once or twice she had peeped into the book her sister was reading but it had no pictures or conversations in it and what is the use of a book thought Alice without pictures or conversations  So she was considering in her own mind as well as she could for the hot day made her feel very sleepy and stupid whether the pleasure of making a daisychain would be worth the trouble of getting up and picking the daisies when suddenly a White Rabbit with pink eyes ran close by her.  There was nothing so very remarkable in that nor did Alice think it so very much out of the way to hear the Rabbit say to itself Oh dear Oh dear I shall be late when she thought it over afterwards it occurred to her that she ought to have wondered at this but at the time it all seemed quite natural but when the Rabbit actually took a watch out of its waistcoatpocket and looked at it a

In [ ]:
data_for_verbs_list = data_for_verbs.split(sep='.')
data_for_verbs_list

['CHAPTER I',
 ' Down the RabbitHole   Alice was beginning to get very tired of sitting by her sister on the bank and of having nothing to do once or twice she had peeped into the book her sister was reading but it had no pictures or conversations in it and what is the use of a book thought Alice without pictures or conversations  So she was considering in her own mind as well as she could for the hot day made her feel very sleepy and stupid whether the pleasure of making a daisychain would be worth the trouble of getting up and picking the daisies when suddenly a White Rabbit with pink eyes ran close by her',
 '  There was nothing so very remarkable in that nor did Alice think it so very much out of the way to hear the Rabbit say to itself Oh dear Oh dear I shall be late when she thought it over afterwards it occurred to her that she ought to have wondered at this but at the time it all seemed quite natural but when the Rabbit actually took a watch out of its waistcoatpocket and looke

In [ ]:
final_sentences = []
for i in data_for_verbs_list:
  i = i.lower()
  i = " ".join(i.split())
  if 'alice' in i:
    tokens = TreebankWordTokenizer().tokenize(i)
    stop_words = stopwords.words("english") 
    tokens = [token for token in tokens if token not in stop_words]
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    text = " ".join(tokens)
    i = text
    final_sentences.append(i)
final_sentences

['rabbithole alice beginning get tired sitting sister bank nothing twice peeped book sister reading picture conversation use book thought alice without picture conversation considering mind well could hot day made feel sleepy stupid whether pleasure making daisychain would worth trouble getting picking daisy suddenly white rabbit pink eye ran close',
 'nothing remarkable alice think much way hear rabbit say oh dear oh dear shall late thought afterwards occurred ought wondered time seemed quite natural rabbit actually took watch waistcoatpocket looked hurried alice started foot flashed across mind never seen rabbit either waistcoatpocket watch take burning curiosity ran across field fortunately time see pop large rabbithole hedge',
 'another moment went alice never considering world get',
 'rabbithole went straight like tunnel way dipped suddenly suddenly alice moment think stopping found falling deep well',
 'well thought alice fall shall think nothing tumbling stair brave theyll think

In [ ]:
verbs = {}
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
for i in final_sentences:
  tokens = word_tokenize(i)
  verbs_in_sentence = [word for (word, pos) in nltk.pos_tag(tokens) if pos.startswith('V')]
  for j in verbs_in_sentence:
    if j not in verbs.keys():
      verbs[j] = 1
    else:
      verbs[j] += 1
sorted_verbs = dict(sorted(verbs.items(), key=lambda item: item[1], reverse=True))
print(list(sorted_verbs.items())[0])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


('said', 210)
